In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Aug  3 13:02:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [4]:
# fashion_mnist = tf.keras.datasets.fashion_mnist
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# X_train = X_train.astype(dtype) / 255.0
# y_train = y_train.astype(dtype)
# X_test = X_test.astype(dtype)  / 255.0
# y_test = y_test.astype(dtype)

# X_train = np.reshape(X_train, (-1, 784))
# X_test = np.reshape(X_test, (-1, 784))

In [5]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(dtype) / 255.0
y_train = y_train.astype(dtype)
X_test = X_test.astype(dtype)  / 255.0
y_test = y_test.astype(dtype)

X_train = np.reshape(X_train, (-1, 3072))
X_test = np.reshape(X_test, (-1, 3072))

170500096/170498071 [==============================] - 25s 0us/step


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [11]:
class SSRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, regularization_penalty, regularization_method):
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method

    def __call__(self, x):
        if self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # The scaling vector could be [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        # and the resulting weighted values could be
        #
        # [[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_vector = tf.cumsum(tf.constant(self.regularization_penalty, shape=(x.shape[-1],), dtype=dtype), axis=0)
        weighted_values = scaling_vector * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


class SSLayer(tf.keras.Model):
    def __init__(self, input_units, units, activation, regularization_penalty, regularization_method, kernel_initializer, bias_initializer, regularize=True):
        super().__init__()

        self.input_units = input_units
        self.units = units
        self.activation = activation
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = SSRegularizer(self.regularization_penalty, self.regularization_method)
        
        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(units,), dtype=dtype),
            trainable=True)
        
        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        copy = SSLayer(
            self.input_units, 
            self.units, 
            self.activation, 
            regularization_penalty=self.regularization_penalty, 
            regularization_method=None, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer
        )
        copy.W = self.W
        copy.b = self.b
        return copy


class SSModel(tf.keras.Model):
    def __init__(self, layer_sizes, activation=None, regularization_penalty=0.01, regularization_method='weighted_l1', kernel_initializer='glorot_uniform', bias_initializer='zeros'):
        super().__init__()
        
        self.sslayers = list()
        for l in range(len(layer_sizes) - 1):
            input_units = layer_sizes[l]
            units = layer_sizes[l + 1]
            if l < len(layer_sizes) - 2:
                layer = SSLayer(input_units, units, activation, regularization_penalty, regularization_method, kernel_initializer, bias_initializer)
            else:  # Last layer
                layer = SSLayer(input_units, units, 'softmax', 0., regularization_method, kernel_initializer, bias_initializer)
            self.sslayers.append(layer)

    def call(self, inputs):
        x = inputs
        for layer in self.sslayers:
            x = layer(x)
        return x
    
    def get_layer_sizes(self):
        layer_sizes = list()
        for l in range(len(self.sslayers)):
            layer = self.sslayers[l]
            layer_sizes.append(layer.W.shape[0])
            if l == len(self.sslayers) - 1:  # Last layer
                layer_sizes.append(layer.W.shape[1])
        return layer_sizes
    
    def print_neurons(self):
        for layer in self.sslayers[:-1]:
            print(get_param_string(layer.W, layer.b))
    
    def remove_regularization(self):
        for l in range(len(self.sslayers)):
            self.sslayers[l] = self.sslayers[l].copy_without_regularization()
    
    def get_regularization_penalty(self):
        return self.sslayers[0].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for l in range(0, len(self.sslayers) - 1):  # Every layer except of the last is regularized
            self.sslayers[l].regularizer.regularization_penalty = regularization_penalty
    
    def prune(self, threshold=0.001):
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
            
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            weights_with_biases = tf.concat([W1, tf.reshape(b1, (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
            new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
            new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

            layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
    
    def grow(self, min_new_neurons=5, scaling_factor=0.001):   
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
       
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            n_new_neurons = max(min_new_neurons, int(W1.shape[1] * 0.2))

            W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
            b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
            W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.

            new_W1 = tf.concat([W1, W1_growth], axis=1)
            new_b1 = tf.concat([b1, b1_growth], axis=0)
            new_W2 = tf.concat([W2, W2_growth], axis=0)

            layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)

In [24]:
def get_param_string(weights, bias):
    param_string = ""
    weights_with_bias = tf.concat([weights, tf.reshape(bias, (1, -1))], axis=0)
    max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
    magnitudes = np.floor(np.log10(max_parameters))
    for m in magnitudes:
        if m > 0:
            m = 0
        param_string += str(int(-m))
    return param_string


def print_epoch_statistics(model, x, y, validation_data):
    x_val = validation_data[0]
    y_val = validation_data[1]

    y_pred = model(x)
    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y, y_pred))
    accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y, y_pred))
    
    y_val_pred = model(x_val)
    val_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_val, y_val_pred))
    val_accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y_val, y_val_pred))
    print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {model.get_regularization_penalty()}")
    print(f"layer sizes: {model.get_layer_sizes()}")
    # model.print_neurons()

    return val_loss


def train_model(model, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, regularization_penalty_multiplier=1.):
    train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    best_val_loss = np.inf
    training_stalled = False
    for epoch in range(epochs):
        print("##########################################################")
        print(f"Epoch {epoch + 1}/{epochs}")

        if epoch < self_scaling_epochs:
            print("Before growing:")

            val_loss = print_epoch_statistics(model, x, y, validation_data)
            if val_loss >= best_val_loss:
                if not training_stalled:
                    penalty = model.get_regularization_penalty() * regularization_penalty_multiplier
                    model.set_regularization_penalty(penalty)
                    training_stalled = True
            else:
                best_val_loss = val_loss
                training_stalled = False

            model.grow(min_new_neurons=min_new_neurons, scaling_factor=0.001)
            print("After growing:")
            print_epoch_statistics(model, x, y, validation_data)
        
        if epoch == self_scaling_epochs:
            model.remove_regularization()

        for step, (x_batch, y_batch) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                y_pred = model(x_batch, training=True)
                loss_value = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
                loss_value += sum(model.losses)

            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if epoch < self_scaling_epochs:
            print("Before pruning:")
            print_epoch_statistics(model, x, y, validation_data)
            model.prune(threshold=0.001)
            print("After pruning:")
            print_epoch_statistics(model, x, y, validation_data)
        else:
            print_epoch_statistics(model, x, y, validation_data)

# Efficient training

In [9]:
epochs = 10
self_scaling_epochs = 10
batch_size = 32
min_new_neurons = 20

In [16]:
%%time

model = SSModel(layer_sizes=[3072, 1000, 1000, 1000, 1000, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

Epoch 1/10
Before growing:
loss: 2.7915027141571045 - accuracy: 0.11311999708414078 - val_loss: 2.766880750656128 - val_accuracy: 0.11949999630451202
layer sizes: [3072, 1000, 1000, 1000, 1000, 10]
2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

In [17]:
%%time

model = SSModel(layer_sizes=[3072, 1000, 1000, 1000, 1000, 10], activation='selu', regularization_penalty=0.00001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

Epoch 1/10
Before growing:
loss: 2.701932430267334 - accuracy: 0.09616000205278397 - val_loss: 2.6937167644500732 - val_accuracy: 0.10040000081062317
layer sizes: [3072, 1000, 1000, 1000, 1000, 10]
2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

In [18]:
%%time

model = SSModel(layer_sizes=[3072, 1000, 1000, 1000, 1000, 10], activation='selu', regularization_penalty=0.000001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

Epoch 1/10
Before growing:
loss: 2.662282705307007 - accuracy: 0.1032399982213974 - val_loss: 2.657341718673706 - val_accuracy: 0.10700000077486038
layer sizes: [3072, 1000, 1000, 1000, 1000, 10]
222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

In [25]:
epochs = 50
self_scaling_epochs = 50
batch_size = 32
min_new_neurons = 50

In [26]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1)

##########################################################
Epoch 1/50
Before growing:
loss: 2.9734561443328857 - accuracy: 0.0878399983048439 - val_loss: 2.957174301147461 - val_accuracy: 0.0892999991774559 - penalty: 0.0001
layer sizes: [3072, 300, 300, 300, 300, 10]
After growing:
loss: 2.973456621170044 - accuracy: 0.0878399983048439 - val_loss: 2.957174301147461 - val_accuracy: 0.0892999991774559 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
Before pruning:
loss: 1.8268446922302246 - accuracy: 0.3210600018501282 - val_loss: 1.8327789306640625 - val_accuracy: 0.3165000081062317 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
After pruning:
loss: 1.8263518810272217 - accuracy: 0.32137998938560486 - val_loss: 1.8324464559555054 - val_accuracy: 0.31679999828338623 - penalty: 0.0001
layer sizes: [3072, 15, 58, 34, 70, 10]
##########################################################
Epoch 2/50
Before growing:
loss: 1.8263518810272217 - accuracy: 0.321379989385

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1)

##########################################################
Epoch 1/50
Before growing:
loss: 2.72518253326416 - accuracy: 0.10886000096797943 - val_loss: 2.734980821609497 - val_accuracy: 0.10670000314712524 - penalty: 0.0001
layer sizes: [3072, 300, 300, 300, 300, 10]
After growing:
loss: 2.72518253326416 - accuracy: 0.10886000096797943 - val_loss: 2.734980821609497 - val_accuracy: 0.10670000314712524 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
Before pruning:
loss: 1.8462209701538086 - accuracy: 0.3119400143623352 - val_loss: 1.8498504161834717 - val_accuracy: 0.30640000104904175 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
After pruning:
loss: 1.8453584909439087 - accuracy: 0.3140600025653839 - val_loss: 1.848866581916809 - val_accuracy: 0.30880001187324524 - penalty: 0.0001
layer sizes: [3072, 13, 62, 33, 64, 10]
##########################################################
Epoch 2/50
Before growing:
loss: 1.8453584909439087 - accuracy: 0.314060002565

## Keras

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3072, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(1108, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(1108, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(1108, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(1108, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(10, activation='softmax', kernel_initializer='lecun_normal'),
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

In [ ]:
%%time

model.fit(X_train_norm, y_train, epochs=15, validation_data=(X_test_norm, y_test))

Epoch 1/15
1563/1563 [==============================] - 7s 4ms/step - loss: 1.9114 - accuracy: 0.3848 - val_loss: 1.4940 - val_accuracy: 0.4722
Epoch 2/15
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4642 - accuracy: 0.4813 - val_loss: 1.4827 - val_accuracy: 0.4778
Epoch 3/15
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3553 - accuracy: 0.5187 - val_loss: 1.4176 - val_accuracy: 0.5004
Epoch 4/15
1563/1563 [==============================] - 7s 4ms/step - loss: 1.2516 - accuracy: 0.5545 - val_loss: 1.3573 - val_accuracy: 0.5251
Epoch 5/15
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1249 - accuracy: 0.5983 - val_loss: 1.4309 - val_accuracy: 0.5205
Epoch 6/15
1563/1563 [==============================] - 7s 4ms/step - loss: 1.0061 - accuracy: 0.6386 - val_loss: 1.4089 - val_accuracy: 0.5283
Epoch 7/15
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8637 - accuracy: 0.6877 - val_loss: 1.4936 - val_accuracy: